# COLLECTING DATA
---

Nhiệm vụ của phần này là thực hiện lấy dữ liệu từ thông tin imdbid và title trong file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [14]:
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup
import re

## 2. HÀM LẤY DỮ LIỆU CỦA TRANG `OMDB API`

Phần dưới này để đổi thông tin người thực hiện

In [8]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None, # đk api xong áp vô nè
        "start": 1259, # 
        "end": 1887
    },

    "Thi": {
        "part": 2,
        "api_key": None, # đk api xong áp vô nè
        "start": 629,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec',
        "start": 0,
        "end": 628
    } 
}

In [9]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]
start = info["start"]
end = info["end"]

Hàm này trả về link từ id hoặc tiêu đề phim

In [12]:
def convert_url(item: str, key = 0):
    if item.startswith("tt"):
        url = f'https://www.omdbapi.com/?i={item}&apikey={key}'
    else: 
        title = item.split(' __ ')[0].split(":")[-1].strip()
        title = re.sub(r'[^a-zA-Z0-9\s]', '', title)
            
        year = item.split(' __ ')[1]
        title = '+'.join(title.split(' '))
        url = f'https://www.omdbapi.com/?t={title}&y={year}&apikey={key}'

    return url

Hàm này thì thu thập dữ liệu

In [24]:
def collectMovieData(imdbIds_titles, key = apiKey, start = start, end = end):
    title = []
    rated = []
    released = []
    runtime = []
    genre = []
    metaScore = []
    rating = []
    vote = []
    boxCffice = []
    language = []
    award = []

    for item in imdbIds_titles[start:end]:
        print(f"[LOG]: Collecting data of {item}")
        url = convert_url(item, key=key)
        trying = 1

        while True:    
            respond = requests.get(url)
            if respond.status_code == 200:
                data = respond.json()
                if data.get("Response") == "False": # bỏ qua những phim không thể tim thấy trong trang
                    break

                title.append(data['Title'])
                rated.append(data['Rated'])
                released.append(data['Released'])
                runtime.append(data['Runtime'])
                genre.append(data['Genre'])
                metaScore.append(data['Metascore'])
                rating.append(data['imdbRating'])
                vote.append(data['imdbVotes'])
                boxCffice.append(data['BoxOffice'])
                language.append(data['Language'])
                award.append(data['Awards'])
                break
                
            else:
                trying += 1
                if trying > 3:
                    print(f"Request fail after 3 retries with code {respond.status_code}")

                time.sleep(1)

    movieDataframe = pd.DataFrame({
            "Title": title,
            "Rated": rated,
            "Lanuage": language,
            "Released": released,
            "Runtime": runtime,
            "Genre": genre,
            "Awards": award,
            "Metascore": metaScore,
            "IMDb Rating": rating,
            "IMDb Votes": vote,
            "Domestic Revenue": boxCffice
        })

    return movieDataframe

Phần này lấy các title và id

In [19]:
ids_titles = []

with open('../Data/ids_titles.csv', 'r') as f:
    ids_titles = [line.strip() for line in f]

này là thu thập dữ liệu nè

In [25]:
df1 = collectMovieData(ids_titles, key=apiKey, start=0, end= 628)
df1

[LOG]: Collecting data of tt28097834
[LOG]: Collecting data of tt10651230
[LOG]: Collecting data of The Burial __ 2023
[LOG]: Collecting data of tt21807222
[LOG]: Collecting data of tt23334616
[LOG]: Collecting data of tt15140278
[LOG]: Collecting data of tt3715152
[LOG]: Collecting data of tt21638298
[LOG]: Collecting data of tt26938504
[LOG]: Collecting data of tt15783426


,Title,Rated,Lanuage,Released,Runtime,Genre,Awards,Metascore,IMDb Rating,IMDb Votes,Domestic Revenue
0,The Mission,PG-13,English,08 Dec 2023,103 min,Documentary,11 nominations,N/A,6.4,"1,075","$58,377"
1,The Peasants,R,"Polish, Latin",13 Oct 2023,114 min,"Animation, Drama, History",10 wins & 19 nominations,N/A,7.6,"6,678","$155,188"
2,The Burial,R,English,13 Oct 2023,126 min,Drama,2 wins & 4 nominations,N/A,7.1,"37,620",N/A
3,Saw X,R,"English, Spanish",29 Sep 2023,118 min,"Horror, Mystery, Thriller",15 nominations,N/A,6.6,"84,301","$53,607,898"
4,Last Call for Istanbul,TV-MA,Turkish,24 Nov 2023,91 min,"Drama, Romance",1 win,N/A,5.3,"9,358",N/A
5,Coup de Chance,PG-13,French,05 Apr 2024,93 min,"Comedy, Crime, Drama",N/A,N/A,6.4,"8,274",N/A
6,Shooting Stars,PG-13,English,02 Jun 2023,116 min,"Biography, Drama, History",2 wins & 3 nominations,62,6.5,"2,973",N/A
7,Wingwomen,TV-MA,French,01 Nov 2023,116 min,"Action, Comedy, Crime",N/A,N/A,5.7,"7,003",N/A
8,Love in the Maldives,TV-G,English,01 Apr 2023,84 min,Romance,N/A,N/A,6.1,949,N/A
9,One Day as a Lion,R,English,07 Apr 2023,87 min,"Action, Comedy, Thriller",N/A,N/A,5.2,"2,088","$42,338"


## 3. LƯU DỮ LIỆU THÔ VÀO FILE CSV

In [ ]:
file_name = os.path.join("./Data/", f"raw_movie_data_{str(part)}.csv")
df1.to_csv(file_name)